# Setup


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

DATA_DIR = Path().cwd() / "data"

plt.style.use('ggplot')

# Embeddings


# Counting the number of revisions over time
